In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI 
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature = 0.1)





In [16]:
examples = [
    {
        "movie": "Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "movie": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "Antman",
        "answer": """
        🐜🦸🔬
        """,
    },
]

In [70]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human","What do you know about {movie}?"),
    ("ai", "{answer}")
])
example_select_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

memory = ConversationSummaryBufferMemory( 
    llm = llm,
    max_token_limit= 150,
    return_messages = True,  ## 문자열로 바꾸지 말고 실제 메세지로 해달라고 하는 것을 의미.
    memory_key="chat_history",
    )


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["chat_history"]


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "If AI receive movie name question, AI should 3 imoji answer. But If question is not movie name, AI talk to human without imoji"),
        MessagesPlaceholder(variable_name="chat_history"),
        example_select_prompt,
        ("human", "{question}"),
    ]
)

chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)  

In [71]:
invoke_chain("movie: Ironman")

{'question': 'movie: Ironman'}
content='\n        🦾🔥💰'


In [72]:
invoke_chain("movie:Avengers")

{'question': 'movie:Avengers'}
content='\n        🦸\u200d♂️🦸\u200d♀️🌍'


In [73]:
invoke_chain("What is my first question?")

{'question': 'What is my first question?'}
content='I believe your first question was about the movie "Ironman."'
